# Преобразования данных о недвижимости

В прошлом модуле мы совершили множество преобразований над нашей таблицей — давайте вспомним их:

*  удалили столбцы `index` и `Coordinates`;
*  создали признак средней площади одной комнаты `MeanRoomsSquare` и ввели коэффициент соотношения площади здания к площади участка — `AreaRatio`;
* преобразовали признак даты продажи `Date` в формат `datetime` и создали на его основе следующие столбцы: номер месяц продажи (`MonthSale`), номер дня недели продажи (`WeekdaySale`), «признак-мигалку» выходного дня (`Weekend`);
* заменили признак года постройки здания `YearBuilt` на его возраст `AgeBuilding`;
извлекли из признака адреса объекта `Address` новый признак подтипа улицы `StreetType` и удалили столбец с адресом;
уменьшили число уникальных наименований агентств по недвижимости (`SellerG`), а также число пригородов (`Suburb`);
выделили категориальные признаки и преобразовали их в тип данных `сategory`;
* заменили сокращённые названия категорий признака типа объекта `Type` на их полные названия (h — house, t —  townhouse, u — unit).

# 1.1

Преобразуйте столбец `Date` в формат `datetime` и выделите квартал (`quarter`) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [1]:
import pandas as pd

melb_df = pd.read_csv("data/melb_data_fe.csv")
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
print(melb_df.Date.dt.quarter.value_counts())

3    4873
2    4359
4    2329
1    2019
Name: Date, dtype: int64


# 1.2

Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы `Date`, `Rooms`, `Bedroom`, `Bathroom`, `Car`.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных `category`.

In [2]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
max_unique_count = 150 
for col in melb_df.columns: 
    if (melb_df[col].nunique() < max_unique_count and 
        col not in cols_to_exclude): 
        melb_df[col] = melb_df[col].astype('category')
        
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

# Сортировка

`sort_values()` - метод

Параметры:

* `by` - имя столбца или список столбцов, по которым производится сортировка;

*  `axis=0` ось по которой производится сортировка (по умолчанию по строкам = 0);

* `ascending=True` - от меньшего к большему;

* `ingnore_index=False` - сохраняет исходные индексы таблицы;

* `inplace=False` - сортировать на месте (изменение исходной) или вернуть сортированную копию.

Комбинирование сортировки с фильтрацией

Найдём информацию о таунхаусах (`Type`), проданных компанией (`SellerG`) `McGrath`, у которых коэффициент соотношения площадей здания и участка (`AreaRatio`) меньше -0.8 (у которых площадь участка существенно больше площади здания). Результат отсортируем по дате продажи (`Date`) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр `ignore_index` на `True`. Для наглядности результата выберем из таблицы только столбцы `Data` и `AreaRatio`:



In [3]:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

,Date,AreaRatio
0,2016-07-26,-0.974922
1,2016-09-24,-0.971831
2,2016-11-27,-0.953608
3,2016-12-11,-0.945946
4,2017-08-04,-0.947368
5,2017-08-04,-0.970874


# 2.2

Произведите сортировку столбца `AreaRatio` по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке `1558`? Ответ округлите до целого числа.



In [4]:
melb_df.sort_values(by='AreaRatio', ascending=False, ignore_index=True).loc[1558, 'BuildingArea']

126.0

# 2.3

Найдите таунхаусы (`Type`) с количеством жилых комнат (`Rooms`) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (`MeanRoomsSquare`). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

In [5]:
mask1 = melb_df.Type == 'townhouse'
mask2 = melb_df.Rooms > 2
melb_df[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price']

1300000.0

# Группировка

Метод `groupby()`

Параметры метода:

* `by` — имя или список имён столбцов, по которым производится группировка.
* `axis` — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
* `as_index` — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на `True`.

возвращает объект `DataFrameGroupBy`, который хранит в себе информацию о том, какие строки относятся к определённой группе.

к этому объекту можно применять агрегирующие методы (`mean`, `median`, `sum` и т. д.), чтобы рассчитывать показатели внутри каждой группы.



In [6]:
# Пример: удаленность района от центра

melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

Метод `agg()`

используется для расчета нескольких агрегирующих методов.

In [7]:
# пример использования метода agg

melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

,count,mean,max
MonthSale,,,
8,1850,1.056371e+06,6500000.0
7,1835,9.314698e+05,9000000.0
5,1644,1.097807e+06,8000000.0
6,1469,1.068981e+06,7650000.0
3,1408,1.146762e+06,5600000.0
4,1246,1.050479e+06,5500000.0
9,1188,1.126349e+06,6400000.0
10,854,1.135970e+06,6250000.0
11,750,1.142503e+06,5050000.0


In [8]:
# полная информация обо всех статистических методах

melb_df.groupby('MonthSale')['Price'].agg('describe')

,count,mean,std,min,25%,50%,75%,max
MonthSale,,,,,,,,
1,278.0,9.397921e+05,577668.924214,170000.0,570500.0,795000.0,1111250.0,5200000.0
2,333.0,1.169051e+06,671564.357417,131000.0,710000.0,1020000.0,1478000.0,4735000.0
3,1408.0,1.146762e+06,709573.596867,85000.0,680000.0,945000.0,1400000.0,5600000.0
4,1246.0,1.050479e+06,591892.902979,145000.0,655000.0,905500.0,1298750.0,5500000.0
5,1644.0,1.097807e+06,668492.867996,145000.0,650000.0,905000.0,1371250.0,8000000.0
6,1469.0,1.068981e+06,606010.069052,222000.0,660000.0,900000.0,1325000.0,7650000.0
7,1835.0,9.314698e+05,537390.803161,190000.0,586750.0,800000.0,1150000.0,9000000.0
8,1850.0,1.056371e+06,619617.476541,160000.0,635000.0,892000.0,1310000.0,6500000.0
9,1188.0,1.126349e+06,608734.690742,170000.0,725000.0,980000.0,1360000.0,6400000.0


Метод `agg` с другими функциями. 

Передадим дополнительно встроенную функцию `set`, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов

In [9]:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{hockingstuart, Biggin, RT, Ray, HAR, Harcourt..."
Eastern Victoria,11,"{McGrath, HAR, hockingstuart, Fletchers, other..."
Northern Metropolitan,40,"{Nick, Greg, Biggin, hockingstuart, RT, Ray, H..."
Northern Victoria,11,"{YPA, McGrath, McDonald, Buckingham, HAR, hock..."
South-Eastern Metropolitan,25,"{hockingstuart, Greg, Biggin, O'Brien, Ray, HA..."
Southern Metropolitan,38,"{Nick, hockingstuart, Greg, Biggin, RT, O'Brie..."
Western Metropolitan,34,"{Greg, hockingstuart, Biggin, RT, O'Brien, HAR..."
Western Victoria,6,"{YPA, hockingstuart, Ray, other, HAR, Raine}"


# 3.1

Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

In [28]:
melb_df.groupby(by='Rooms')['Price'].mean().sort_values(
    ascending=False).index[0]

7

# 3.2

Какой регион имеет наименьшее стандартное отклонение по географической широте (`Lattitude`)?
В качестве ответа запишите название этого региона.

In [29]:
melb_df.groupby(by='Regionname')['Lattitude'].std().sort_values().index[0]

'Western Victoria'

# 3.3

Какая риелторская компания (`SellerG`) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
Для ответа на этот вопрос рассчитайте сумму продаж (`Price`) каждой компании в заданный период.
Не забудьте перевести даты в формат `datetime`.

In [49]:
mask1 = melb_df['Date'] <= pd.to_datetime('2017-09-01') 
mask2 = melb_df['Date'] >= pd.to_datetime('2017-05-01')
melb_df[mask1 & mask2].groupby(by='SellerG')['Price'].sum().sort_values().index[0]


'LITTLE'